This is our best scoring model, which got 0.84517 in public scoring

In [ ]:
# model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# ------------------------------------------------------------------------
#                       Squeeze-and-Excitation Block
# ------------------------------------------------------------------------
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block:
    - Reduces channel dimensions for global information capture.
    - Scales input features based on their importance.
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze: Global average pooling across spatial dimensions
        squeeze = x.view(b, c, -1).mean(dim=2)
        # Excitation: Fully connected layers with activation
        excitation = F.relu(self.fc1(squeeze))
        excitation = torch.sigmoid(self.fc2(excitation)).view(b, c, 1, 1)
        return x * excitation  # Scale input features


# ------------------------------------------------------------------------
#                       Stochastic Depth
# ------------------------------------------------------------------------
class StochasticDepth(nn.Module):
    """
    Stochastic Depth (also known as DropPath):
    - Randomly drops residual branches during training for better regularization.
    """
    def __init__(self, p: float = 0.1):
        super().__init__()
        self.p = p

    def forward(self, x, residual):
        if not self.training or self.p == 0.0:
            return x + residual  # No drop during evaluation
        if torch.rand(1).item() < self.p:
            return x  # Drop residual branch
        else:
            return x + residual  # Keep residual branch


# ------------------------------------------------------------------------
#       Pre-Activation Residual Block with SE + Stochastic Depth
# ------------------------------------------------------------------------
class PreActBlock(nn.Module):
    """
    Pre-Activation Residual Block:
    - Uses BatchNorm and ReLU before convolution layers.
    - Incorporates SE and Stochastic Depth for enhanced performance.
    """
    def __init__(self, in_planes, out_planes, stride=1, drop_prob=0.0):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.se = SEBlock(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)

        # Handle shortcut connections for downsampling
        self.shortcut = None
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Conv2d(in_planes, out_planes, kernel_size=1,
                                      stride=stride, padding=0, bias=False)

        self.sd = StochasticDepth(p=drop_prob)  # Stochastic Depth probability

    def forward(self, x):
        out = F.relu(self.bn1(x))
        out = self.se(out)  # Apply SE block
        shortcut = x if self.shortcut is None else self.shortcut(out)
        out = self.conv1(out)
        out = F.relu(self.bn2(out))
        out = self.conv2(out)
        return self.sd(shortcut, out)  # Apply Stochastic Depth


# ------------------------------------------------------------------------
#       LightResNet-18 for CIFAR-10 (Initial Channels=32)
# ------------------------------------------------------------------------
class LightResNet18(nn.Module):
    """
    A lightweight ResNet-18 architecture designed for CIFAR-10:
    - Uses 32 initial channels instead of 64 for reduced parameters.
    - Incorporates SE and Stochastic Depth for improved generalization.
    """
    def __init__(self, num_classes=10, drop_prob=0.1):
        super().__init__()
        self.in_planes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1,
                               padding=1, bias=False)

        # Create layers with increasing channels and dropout probability
        self.layer1 = self._make_layer(32, 2, stride=1, base_p=drop_prob*1/4)
        self.layer2 = self._make_layer(64, 2, stride=2, base_p=drop_prob*2/4)
        self.layer3 = self._make_layer(128, 2, stride=2, base_p=drop_prob*3/4)
        self.layer4 = self._make_layer(256, 2, stride=2, base_p=drop_prob*4/4)

        self.bn = nn.BatchNorm2d(256)
        self.linear = nn.Linear(256, num_classes)

        # Weight Initialization for Convolution, BatchNorm, and Linear layers
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, out_planes, blocks, stride, base_p):
        """Creates a series of PreActBlocks with increasing dropout probability."""
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for i in range(blocks):
            block_p = base_p * (i+1)/blocks  # Gradually increase drop probability
            layers.append(PreActBlock(self.in_planes, out_planes,
                                      stride=strides[i],
                                      drop_prob=block_p))
            self.in_planes = out_planes  # Update channel size
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.relu(self.bn(out))
        out = F.adaptive_avg_pool2d(out, 1).view(out.size(0), -1)
        out = self.linear(out)
        return out


# Quick parameter check for debugging
if __name__ == "__main__":
    model = LightResNet18(num_classes=10, drop_prob=0.1)
    x = torch.randn(2, 3, 32, 32)
    y = model(x)
    print("Output shape:", y.shape)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,} (~{total_params/1e6:.2f}M)")

Output shape: torch.Size([2, 10])
Total trainable parameters: 2,811,192 (~2.81M)


In [ ]:
# train.py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import pickle
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#from model import LightResNet18  # Assuming this model is defined elsewhere
from tqdm import tqdm  # For progress bars

# Helper function for CutMix to generate random bounding box coordinates
def rand_bbox(size, lam):
    """Generate random bounding box coordinates for CutMix augmentation.
    Args:
        size: Tensor size [batch_size, channels, height, width]
        lam: Lambda value from beta distribution
    Returns:
        Tuple of (bbx1, bby1, bbx2, bby2) coordinates
    """
    W = size[3]
    H = size[2]
    cut_rat = np.sqrt(1. - lam)  # Calculate cut ratio
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # Center coordinates
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # Bounding box coordinates with clipping to stay within image dimensions
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def mixup_data(x, y, alpha=1.0):
    """Applies MixUp augmentation to a batch of data.
    Args:
        x: Input images tensor
        y: Target labels tensor
        alpha: Parameter for beta distribution
    Returns:
        Mixed images, labels_a, labels_b, and lambda value
    """
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size)  # Random permutation of indices
    mixed_x = lam * x + (1 - lam) * x[index, :]  # Linear combination of images
    y_a, y_b = y, y[index]  # Original and permuted labels
    return mixed_x, y_a, y_b, lam

def cutmix_data(x, y, alpha=1.0):
    """Applies CutMix augmentation to a batch of data.
    Args:
        x: Input images tensor
        y: Target labels tensor
        alpha: Parameter for beta distribution
    Returns:
        Modified images, labels_a, labels_b, and adjusted lambda
    """
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size)

    # Generate random bounding box coordinates
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)

    # Replace selected region with another image's region
    x[:, :, bby1:bby2, bbx1:bbx2] = x[index, :, bby1:bby2, bbx1:bbx2]

    # Adjust lambda based on the area of the cut region
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

def mixup_cutmix_collate(batch, alpha=1.0, p=0.5):
    """Custom collate function that randomly applies MixUp or CutMix.
    Args:
        batch: Input batch from DataLoader
        alpha: Beta distribution parameter
        p: Probability of choosing MixUp vs CutMix
    Returns:
        Augmented batch with modified labels
    """
    # Unpack the batch
    images, labels = list(zip(*batch))
    images = torch.stack(images, 0)
    labels = torch.tensor(labels, dtype=torch.long)

    # Randomly select between MixUp and CutMix
    if np.random.rand() < p:
        return mixup_data(images, labels, alpha) + ('mixup',)
    else:
        return cutmix_data(images, labels, alpha) + ('cutmix',)

def unpickle(file):
    """Load CIFAR-10 data from pickle file."""
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARDataset(Dataset):
    """Custom Dataset class for CIFAR-10 with optional transformations."""
    def __init__(self, data, labels, transform=None):
        self.data = data.reshape(-1, 3, 32, 32).astype("float32") / 255.0
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert numpy array to PIL Image
        img = self.data[idx]*255
        img = img.astype("uint8")
        img = Image.fromarray(img.transpose(1,2,0))  # Change to HWC format
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)
        return img, label

def load_cifar10_batches(root_dir):
    """Load and concatenate all CIFAR-10 training batches."""
    data_list, labels_list = [], []
    for i in range(1, 6):  # Load all 5 training batches
        batch_file = f"{root_dir}/data_batch_{i}"
        batch = unpickle(batch_file)
        data_list.append(batch[b'data'])
        labels_list.extend(batch[b'labels'])
    X = np.concatenate(data_list, axis=0)
    y = np.array(labels_list)
    return X, y

def train():
    # Device configuration
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 1) Load and split CIFAR-10 dataset
    root_dir = "/content/cifar-10-python/cifar-10-batches-py"
    X, y = load_cifar10_batches(root_dir)

    # Create train/validation split (5000 validation samples)
    total_size = len(X)  # 50000 total samples
    val_size = 5000
    train_size = total_size - val_size
    indices = np.arange(total_size)
    np.random.shuffle(indices)
    train_idx = indices[:train_size]
    val_idx   = indices[train_size:]

    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]

    # 2) Define data transformations
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandAugment(),  # Random augmentation policy
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),  # CIFAR-10 mean
        (0.2470, 0.2435, 0.2616))   # CIFAR-10 std
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
        (0.2470, 0.2435, 0.2616))
    ])

    # Create datasets
    train_dataset = CIFARDataset(X_train, y_train, transform=transform_train)
    val_dataset   = CIFARDataset(X_val,   y_val,   transform=transform_val)

    # 3) Create data loaders with MixUp/CutMix augmentation
    train_loader = DataLoader(
        train_dataset, batch_size=128,
        shuffle=True, num_workers=4,
        collate_fn=lambda b: mixup_cutmix_collate(b, alpha=1.0, p=0.5)
    )
    val_loader   = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

    # 4) Initialize model
    model = LightResNet18(num_classes=10, drop_prob=0.1).to(device)

    # 5) Define loss function, optimizer, and scheduler
    base_criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Regularization
    optimizer = optim.SGD(
        model.parameters(),
        lr=0.1,
        momentum=0.9,
        nesterov=True,
        weight_decay=5e-4  # L2 regularization
    )
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)  # Cosine decay

    best_acc = 0.0
    num_epochs = 300

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_approx = 0.0
        total_approx = 0

        # Training phase with progress bar
        train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)
        for images, label_info in train_pbar:
            # Unpack MixUp/CutMix information
            y_a, y_b, lam, method = label_info
            images = images.to(device)
            y_a = y_a.to(device)
            y_b = y_b.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)

            # Compute MixUp/CutMix loss
            loss = lam * base_criterion(outputs, y_a) + (1 - lam) * base_criterion(outputs, y_b)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update statistics
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            # Calculate approximate accuracy (weighted by lambda)
            correct_approx += (preds == y_a).sum().item() * lam + (preds == y_b).sum().item() * (1 - lam)
            total_approx += images.size(0)

            train_pbar.set_postfix(loss=f"{loss.item():.3f}")

        # Update learning rate
        scheduler.step()

        # Calculate epoch metrics
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc  = 100.0 * correct_approx / total_approx

        # Validation phase
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, pred = torch.max(outputs, 1)
                val_correct += (pred == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100.0 * val_correct / val_total

        # Print epoch summary
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Approx Train Acc: {epoch_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}%")

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"   [*] Saved new best model: Val Acc = {val_acc:.2f}%")

    # Final output
    print(f"Training complete. Best validation accuracy: {best_acc:.2f}%")
    print("Best model saved as best_model.pth")

if __name__ == "__main__":
    train()

Using device: cuda


Epoch 1/300 | Train Loss: 2.1250, Approx Train Acc: 23.84% | Val Acc: 39.90%
   [*] Saved new best model: Val Acc = 39.90%


Epoch 2/300 | Train Loss: 1.9533, Approx Train Acc: 34.79% | Val Acc: 50.34%
   [*] Saved new best model: Val Acc = 50.34%


Epoch 3/300 | Train Loss: 1.8435, Approx Train Acc: 41.25% | Val Acc: 59.78%
   [*] Saved new best model: Val Acc = 59.78%


Epoch 4/300 | Train Loss: 1.7906, Approx Train Acc: 44.13% | Val Acc: 67.26%
   [*] Saved new best model: Val Acc = 67.26%


...

Epoch 296/300 | Train Loss: 1.2474, Approx Train Acc: 69.38% | Val Acc: 94.88%


Epoch 297/300 | Train Loss: 1.2466, Approx Train Acc: 69.03% | Val Acc: 94.92%


Epoch 298/300 | Train Loss: 1.2715, Approx Train Acc: 67.95% | Val Acc: 94.70%


Epoch 299/300 | Train Loss: 1.2799, Approx Train Acc: 67.47% | Val Acc: 94.88%


Epoch 300/300 | Train Loss: 1.2737, Approx Train Acc: 67.95% | Val Acc: 94.98%
Training complete. Best validation accuracy: 95.12%
Best model saved as best_model.pth


In [ ]:
# inference.py
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle
import numpy as np
from torch.utils.data import DataLoader, Dataset
#from model import LightResNet18
import torch.nn.functional as F

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: shape (N, 32, 32, 3)
        ids: array/list of image IDs
        transform: transforms to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].astype("uint8")  # ensure 0..255
        img_id = self.ids[idx]
        if self.transform:
            img = self.transform(img)
        return img, img_id

def inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 1) Load the best model
    model = LightResNet18(num_classes=10, drop_prob=0.1).to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()

    # 2) Load custom test set .pkl
    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    print("Keys in test_dict:", test_dict.keys())

    # Reshape if it's (N, 3072). If it's already (N, 32,32,3), remove reshape
    test_images = test_dict[b'data'].reshape(-1, 32, 32, 3)
    test_ids = [str(i) for i in range(len(test_images))]

    # 3) Define test transforms
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])

    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # 4) Inference with 2-pass TTA
    predictions = []
    image_ids = []
    with torch.no_grad():
        for imgs, ids in test_loader:
            imgs = imgs.to(device)

            # Pass 1: Normal
            out_normal = model(imgs)
            probs_normal = F.softmax(out_normal, dim=1)

            # Pass 2: Horizontal flip
            imgs_flipped = torch.flip(imgs, dims=[3])  # flip W dimension
            out_flipped = model(imgs_flipped)
            probs_flipped = F.softmax(out_flipped, dim=1)

            # Average probabilities
            final_probs = (probs_normal + probs_flipped) / 2.0
            _, predicted = torch.max(final_probs, 1)

            predictions.extend(predicted.cpu().numpy().tolist())
            image_ids.extend(ids)

    # 5) Save submission
    submission_df = pd.DataFrame({"ID": image_ids, "Labels": predictions})
    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

if __name__ == "__main__":
    inference()


Using device: cuda
Keys in test_dict: dict_keys([b'data', b'ids'])


<ipython-input-5-6365ab4560f6>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))


Submission file saved as submission.csv


In [ ]:
import pandas as pd

# Load and inspect submission file
submission_df = pd.read_csv("submission.csv")
print(submission_df.head())  # Show first few rows
print(submission_df["Labels"].value_counts())  # Show label distribution


   ID  Labels
0   0       6
1   1       1
2   2       8
3   3       6
4   4       9
Labels
3    1127
5    1119
1    1040
8    1021
9    1018
7     969
2     958
4     940
0     912
6     896
Name: count, dtype: int64
